<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Transformers_for_Time_Series_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader

# Custom dataset for time series data
class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_length):
        self.data = data
        self.seq_length = seq_length

    def __len__(self):
        return len(self.data) - self.seq_length

    def __getitem__(self, index):
        x = self.data[index:index + self.seq_length]
        y = self.data[index + self.seq_length]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# Transformer model for time series prediction
class TransformerTimeSeries(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, output_dim):
        super(TransformerTimeSeries, self).__init__()
        self.embedding = nn.Linear(input_dim, model_dim)  # Map input_dim to model_dim
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(model_dim, output_dim)

    def forward(self, src):
        # Apply embedding layer to input data
        src = self.embedding(src)  # [batch_size, seq_len, model_dim]
        src = src.transpose(0, 1)  # Reshape for transformer: [seq_len, batch_size, model_dim]
        src = self.transformer_encoder(src)
        src = src.transpose(0, 1).mean(dim=1)  # Restore to [batch_size, model_dim]
        output = self.fc(src)
        return output

# Generate sine wave data
data = np.sin(np.linspace(0, 100, 1000))  # Example sine wave data
dataset = TimeSeriesDataset(data, seq_length=10)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize model, loss function, and optimizer
model = TransformerTimeSeries(input_dim=1, model_dim=32, num_heads=2, num_layers=3, output_dim=1)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(20):
    epoch_loss = 0
    for x, y in dataloader:
        optimizer.zero_grad()
        y_pred = model(x.unsqueeze(-1))  # Add feature dimension: [batch_size, seq_len, input_dim]
        loss = criterion(y_pred.squeeze(), y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {epoch_loss / len(dataloader):.4f}')

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader

# Custom dataset for time series data
class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_length):
        self.data = data
        self.seq_length = seq_length

    def __len__(self):
        return len(self.data) - self.seq_length

    def __getitem__(self, index):
        x = self.data[index:index + self.seq_length]
        y = self.data[index + self.seq_length]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# Transformer model for time series prediction
class TransformerTimeSeries(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, output_dim):
        super(TransformerTimeSeries, self).__init__()
        self.embedding = nn.Linear(input_dim, model_dim)  # Embedding layer to match input_dim to model_dim
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(model_dim, output_dim)

    def forward(self, src):
        # Add embedding and pass through Transformer Encoder
        src = self.embedding(src)  # [batch_size, seq_len, model_dim]
        src = self.transformer_encoder(src)  # [batch_size, seq_len, model_dim]
        src = src.mean(dim=1)  # Pool over sequence length (mean pooling)
        output = self.fc(src)  # Final prediction
        return output

# Generate sine wave data
data = np.sin(np.linspace(0, 100, 1000))  # Example sine wave data
seq_length = 10
dataset = TimeSeriesDataset(data, seq_length=seq_length)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize model, loss function, and optimizer
model = TransformerTimeSeries(input_dim=1, model_dim=32, num_heads=2, num_layers=3, output_dim=1)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(20):
    model.train()  # Set model to training mode
    epoch_loss = 0
    for x, y in dataloader:
        optimizer.zero_grad()
        # Ensure input is correctly shaped for the Transformer
        y_pred = model(x.unsqueeze(-1))  # Add feature dimension: [batch_size, seq_len, input_dim]
        loss = criterion(y_pred.squeeze(), y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {epoch_loss / len(dataloader):.4f}')